### CSC 578 Neural Networks and Deep Learning

### Warmup quiz #1

In [2]:
import numpy as np

In [9]:
"""
network.py
~~~~~~~~~~

A module to implement the stochastic gradient descent learning
algorithm for a feedforward neural network.  Gradients are calculated
using backpropagation.  Note that I have focused on making the code
simple, easily readable, and easily modifiable.  It is not optimized,
and omits many desirable features.
"""

#### Libraries
# Standard library
import random

# Third-party libraries
import numpy as np

class Network(object):

    def __init__(self, sizes):
        """The list ``sizes`` contains the number of neurons in the
        respective layers of the network.  For example, if the list
        was [2, 3, 1] then it would be a three-layer network, with the
        first layer containing 2 neurons, the second layer 3 neurons,
        and the third layer 1 neuron.  The biases and weights for the
        network are initialized randomly, using a Gaussian
        distribution with mean 0, and variance 1.  Note that the first
        layer is assumed to be an input layer, and by convention we
        won't set any biases for those neurons, since biases are only
        ever used in computing the outputs from later layers."""
        self.num_layers = len(sizes)
        self.sizes = sizes
        self.biases = [np.random.randn(y, 1) for y in sizes[1:]]
        self.weights = [np.random.randn(y, x)
                        for x, y in zip(sizes[:-1], sizes[1:])]

    def feedforward(self, a):
        """Return the output of the network if ``a`` is input."""
        for b, w in zip(self.biases, self.weights):
            a = sigmoid(np.dot(w, a)+b)
        return a

    def SGD(self, training_data, epochs, mini_batch_size, eta,
            test_data=None):
        """Train the neural network using mini-batch stochastic
        gradient descent.  The ``training_data`` is a list of tuples
        ``(x, y)`` representing the training inputs and the desired
        outputs.  The other non-optional parameters are
        self-explanatory.  If ``test_data`` is provided then the
        network will be evaluated against the test data after each
        epoch, and partial progress printed out.  This is useful for
        tracking progress, but slows things down substantially."""
        if test_data: n_test = len(test_data)
        n = len(training_data)
        for j in xrange(epochs):
            random.shuffle(training_data)
            mini_batches = [
                training_data[k:k+mini_batch_size]
                for k in xrange(0, n, mini_batch_size)]
            for mini_batch in mini_batches:
                self.update_mini_batch(mini_batch, eta)
            if test_data:
                print "Epoch {0}: {1} / {2}".format(
                    j, self.evaluate(test_data), n_test)
            else:
                print "Epoch {0} complete".format(j)

    def update_mini_batch(self, mini_batch, eta):
        """Update the network's weights and biases by applying
        gradient descent using backpropagation to a single mini batch.
        The ``mini_batch`` is a list of tuples ``(x, y)``, and ``eta``
        is the learning rate."""
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        for x, y in mini_batch:
            delta_nabla_b, delta_nabla_w = self.backprop(x, y)
            nabla_b = [nb+dnb for nb, dnb in zip(nabla_b, delta_nabla_b)]
            nabla_w = [nw+dnw for nw, dnw in zip(nabla_w, delta_nabla_w)]
        self.weights = [w-(eta/len(mini_batch))*nw
                        for w, nw in zip(self.weights, nabla_w)]
        self.biases = [b-(eta/len(mini_batch))*nb
                       for b, nb in zip(self.biases, nabla_b)]

    def backprop(self, x, y):
        """Return a tuple ``(nabla_b, nabla_w)`` representing the
        gradient for the cost function C_x.  ``nabla_b`` and
        ``nabla_w`` are layer-by-layer lists of numpy arrays, similar
        to ``self.biases`` and ``self.weights``."""
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        # feedforward
        activation = x
        activations = [x] # list to store all the activations, layer by layer
        zs = [] # list to store all the z vectors, layer by layer
        for b, w in zip(self.biases, self.weights):
            z = np.dot(w, activation)+b
            zs.append(z)
            activation = sigmoid(z)
            activations.append(activation)
        # backward pass
        delta = self.cost_derivative(activations[-1], y) * \
            sigmoid_prime(zs[-1])
        nabla_b[-1] = delta
        nabla_w[-1] = np.dot(delta, activations[-2].transpose())
        # Note that the variable l in the loop below is used a little
        # differently to the notation in Chapter 2 of the book.  Here,
        # l = 1 means the last layer of neurons, l = 2 is the
        # second-last layer, and so on.  It's a renumbering of the
        # scheme in the book, used here to take advantage of the fact
        # that Python can use negative indices in lists.
        for l in xrange(2, self.num_layers):
            z = zs[-l]
            sp = sigmoid_prime(z)
            delta = np.dot(self.weights[-l+1].transpose(), delta) * sp
            nabla_b[-l] = delta
            nabla_w[-l] = np.dot(delta, activations[-l-1].transpose())
        return (nabla_b, nabla_w)

    def evaluate(self, test_data):
        """Return the number of test inputs for which the neural
        network outputs the correct result. Note that the neural
        network's output is assumed to be the index of whichever
        neuron in the final layer has the highest activation."""
        test_results = [(np.argmax(self.feedforward(x)), y)
                        for (x, y) in test_data]
        return sum(int(x == y) for (x, y) in test_results)

    def cost_derivative(self, output_activations, y):
        """Return the vector of partial derivatives \partial C_x /
        \partial a for the output activations."""
        return (output_activations-y)

#### Miscellaneous functions
def sigmoid(z):
    """The sigmoid function."""
    return 1.0/(1.0+np.exp(-z))

def sigmoid_prime(z):
    """Derivative of the sigmoid function."""
    return sigmoid(z)*(1-sigmoid(z))

In [17]:
"""
mnist_loader
~~~~~~~~~~~~

A library to load the MNIST image data.  For details of the data
structures that are returned, see the doc strings for ``load_data``
and ``load_data_wrapper``.  In practice, ``load_data_wrapper`` is the
function usually called by our neural network code.
"""

#### Libraries
# Standard library
import cPickle
import gzip

# Third-party libraries
import numpy as np

def load_data():
    """Return the MNIST data as a tuple containing the training data,
    the validation data, and the test data.

    The ``training_data`` is returned as a tuple with two entries.
    The first entry contains the actual training images.  This is a
    numpy ndarray with 50,000 entries.  Each entry is, in turn, a
    numpy ndarray with 784 values, representing the 28 * 28 = 784
    pixels in a single MNIST image.

    The second entry in the ``training_data`` tuple is a numpy ndarray
    containing 50,000 entries.  Those entries are just the digit
    values (0...9) for the corresponding images contained in the first
    entry of the tuple.

    The ``validation_data`` and ``test_data`` are similar, except
    each contains only 10,000 images.

    This is a nice data format, but for use in neural networks it's
    helpful to modify the format of the ``training_data`` a little.
    That's done in the wrapper function ``load_data_wrapper()``, see
    below.
    """
    f = gzip.open('../data/mnist.pkl.gz', 'rb')
    training_data, validation_data, test_data = cPickle.load(f)
    f.close()
    return (training_data, validation_data, test_data)

def load_data_wrapper():
    """Return a tuple containing ``(training_data, validation_data,
    test_data)``. Based on ``load_data``, but the format is more
    convenient for use in our implementation of neural networks.

    In particular, ``training_data`` is a list containing 50,000
    2-tuples ``(x, y)``.  ``x`` is a 784-dimensional numpy.ndarray
    containing the input image.  ``y`` is a 10-dimensional
    numpy.ndarray representing the unit vector corresponding to the
    correct digit for ``x``.

    ``validation_data`` and ``test_data`` are lists containing 10,000
    2-tuples ``(x, y)``.  In each case, ``x`` is a 784-dimensional
    numpy.ndarry containing the input image, and ``y`` is the
    corresponding classification, i.e., the digit values (integers)
    corresponding to ``x``.

    Obviously, this means we're using slightly different formats for
    the training data and the validation / test data.  These formats
    turn out to be the most convenient for use in our neural network
    code."""
    tr_d, va_d, te_d = load_data()
    training_inputs = [np.reshape(x, (784, 1)) for x in tr_d[0]]
    training_results = [vectorized_result(y) for y in tr_d[1]]
    training_data = zip(training_inputs, training_results)
    validation_inputs = [np.reshape(x, (784, 1)) for x in va_d[0]]
    validation_data = zip(validation_inputs, va_d[1])
    test_inputs = [np.reshape(x, (784, 1)) for x in te_d[0]]
    test_data = zip(test_inputs, te_d[1])
    return (training_data, validation_data, test_data)

def vectorized_result(j):
    """Return a 10-dimensional unit vector with a 1.0 in the jth
    position and zeroes elsewhere.  This is used to convert a digit
    (0...9) into a corresponding desired output from the neural
    network."""
    e = np.zeros((10, 1))
    e[j] = 1.0
    return e

In [40]:
# Change working directory
import os
#os.getcwd()
os.chdir('/Users/zacklarsen/Desktop/CSC 578 Neural Networks and Deep Learning/neural-networks-and-deep-learning-master/data')
!ls

mnist.pkl.gz


In [41]:
## Load the MNIST data
training_data, validation_data, test_data = \
load_data_wrapper()

In [45]:
## After loading the MNIST data, we'll set up a Network with 30 30 hidden neurons
net = Network([784, 30, 10])

### We'll use stochastic gradient descent to learn from the MNIST training_data over 30 epochs, with a mini-batch size of 10, and a learning rate of η=3.0

In [43]:
net.SGD(training_data, 30, 10, 3.0, test_data=test_data)

Epoch 0: 9130 / 10000
Epoch 1: 9246 / 10000
Epoch 2: 9334 / 10000
Epoch 3: 9353 / 10000
Epoch 4: 9356 / 10000
Epoch 5: 9397 / 10000
Epoch 6: 9412 / 10000
Epoch 7: 9429 / 10000
Epoch 8: 9461 / 10000
Epoch 9: 9436 / 10000
Epoch 10: 9437 / 10000
Epoch 11: 9447 / 10000
Epoch 12: 9443 / 10000
Epoch 13: 9462 / 10000
Epoch 14: 9468 / 10000
Epoch 15: 9449 / 10000
Epoch 16: 9488 / 10000
Epoch 17: 9480 / 10000
Epoch 18: 9507 / 10000
Epoch 19: 9486 / 10000
Epoch 20: 9483 / 10000
Epoch 21: 9479 / 10000
Epoch 22: 9497 / 10000
Epoch 23: 9474 / 10000
Epoch 24: 9493 / 10000
Epoch 25: 9509 / 10000
Epoch 26: 9481 / 10000
Epoch 27: 9494 / 10000
Epoch 28: 9475 / 10000
Epoch 29: 9520 / 10000


### Let's rerun the above experiment, changing the number of hidden neurons to 100

In [44]:
net = Network([784, 100, 10])
net.SGD(training_data, 30, 10, 3.0, test_data=test_data)

Epoch 0: 6586 / 10000
Epoch 1: 6705 / 10000
Epoch 2: 6758 / 10000
Epoch 3: 6770 / 10000
Epoch 4: 6815 / 10000
Epoch 5: 6900 / 10000
Epoch 6: 7736 / 10000
Epoch 7: 7759 / 10000
Epoch 8: 7786 / 10000
Epoch 9: 7773 / 10000
Epoch 10: 7793 / 10000
Epoch 11: 7801 / 10000
Epoch 12: 7795 / 10000
Epoch 13: 7800 / 10000
Epoch 14: 7813 / 10000
Epoch 15: 7811 / 10000
Epoch 16: 7805 / 10000
Epoch 17: 7809 / 10000
Epoch 18: 7828 / 10000
Epoch 19: 7814 / 10000
Epoch 20: 7831 / 10000
Epoch 21: 7812 / 10000
Epoch 22: 7827 / 10000
Epoch 23: 7832 / 10000
Epoch 24: 7831 / 10000
Epoch 25: 7842 / 10000
Epoch 26: 7833 / 10000
Epoch 27: 7836 / 10000
Epoch 28: 7830 / 10000
Epoch 29: 7837 / 10000


### If we choose our hyper-parameters poorly, we can get bad results. Suppose, for example, that we'd chosen the learning rate to be η=0.001

In [46]:
net = Network([784, 100, 10])
net.SGD(training_data, 30, 10, 0.001, test_data=test_data)

Epoch 0: 853 / 10000
Epoch 1: 1074 / 10000
Epoch 2: 1156 / 10000
Epoch 3: 1161 / 10000
Epoch 4: 1167 / 10000
Epoch 5: 1174 / 10000
Epoch 6: 1192 / 10000
Epoch 7: 1216 / 10000
Epoch 8: 1276 / 10000
Epoch 9: 1464 / 10000
Epoch 10: 1935 / 10000
Epoch 11: 1984 / 10000
Epoch 12: 2040 / 10000
Epoch 13: 2115 / 10000
Epoch 14: 2188 / 10000
Epoch 15: 2279 / 10000
Epoch 16: 2381 / 10000
Epoch 17: 2477 / 10000
Epoch 18: 2567 / 10000
Epoch 19: 2642 / 10000
Epoch 20: 2697 / 10000
Epoch 21: 2748 / 10000
Epoch 22: 2791 / 10000
Epoch 23: 2830 / 10000
Epoch 24: 2853 / 10000
Epoch 25: 2896 / 10000
Epoch 26: 2925 / 10000
Epoch 27: 2957 / 10000
Epoch 28: 2991 / 10000
Epoch 29: 3027 / 10000


### The results above are much less encouraging

### Suppose we try the successful 30 hidden neuron network architecture from earlier, but with the learning rate changed to η=100.0

In [48]:
net = Network([784, 30, 10])
net.SGD(training_data, 30, 10, 100.0, test_data=test_data)

Epoch 0: 1063 / 10000
Epoch 1: 1539 / 10000
Epoch 2: 1539 / 10000
Epoch 3: 1533 / 10000
Epoch 4: 1532 / 10000
Epoch 5: 1522 / 10000
Epoch 6: 1527 / 10000
Epoch 7: 1527 / 10000
Epoch 8: 1524 / 10000
Epoch 9: 1513 / 10000
Epoch 10: 1512 / 10000
Epoch 11: 1511 / 10000
Epoch 12: 1506 / 10000
Epoch 13: 1494 / 10000
Epoch 14: 1484 / 10000
Epoch 15: 1484 / 10000
Epoch 16: 1468 / 10000
Epoch 17: 1455 / 10000
Epoch 18: 1417 / 10000
Epoch 19: 1463 / 10000
Epoch 20: 1030 / 10000
Epoch 21: 1030 / 10000
Epoch 22: 1030 / 10000
Epoch 23: 1030 / 10000
Epoch 24: 1030 / 10000
Epoch 25: 1029 / 10000
Epoch 26: 1030 / 10000
Epoch 27: 1030 / 10000
Epoch 28: 1029 / 10000
Epoch 29: 1029 / 10000


### At this point we've actually gone too far, and the learning rate is too high

### Exercise

### Try creating a network with just two layers - an input and an output layer, no hidden layer - with 784 and 10 neurons, respectively. Train the network using stochastic gradient descent. What classification accuracy can you achieve?

In [50]:
net = Network([784, 10])
net.SGD(training_data, 30, 10, 3.0, test_data=test_data)

Epoch 0: 5596 / 10000
Epoch 1: 5746 / 10000
Epoch 2: 6551 / 10000
Epoch 3: 6616 / 10000
Epoch 4: 6648 / 10000
Epoch 5: 6688 / 10000
Epoch 6: 6671 / 10000
Epoch 7: 6676 / 10000
Epoch 8: 6691 / 10000
Epoch 9: 6714 / 10000
Epoch 10: 6730 / 10000
Epoch 11: 6709 / 10000
Epoch 12: 6659 / 10000
Epoch 13: 6742 / 10000
Epoch 14: 6770 / 10000
Epoch 15: 6771 / 10000
Epoch 16: 6766 / 10000
Epoch 17: 6816 / 10000
Epoch 18: 6796 / 10000
Epoch 19: 6768 / 10000
Epoch 20: 6795 / 10000
Epoch 21: 6764 / 10000
Epoch 22: 6799 / 10000
Epoch 23: 6838 / 10000
Epoch 24: 6885 / 10000
Epoch 25: 7557 / 10000
Epoch 26: 7563 / 10000
Epoch 27: 7577 / 10000
Epoch 28: 7574 / 10000
Epoch 29: 7597 / 10000


In [54]:
net = Network([784, 10])
net.SGD(training_data, 30, 10, 2.0, test_data=test_data)

Epoch 0: 5850 / 10000
Epoch 1: 5905 / 10000
Epoch 2: 6796 / 10000
Epoch 3: 7556 / 10000
Epoch 4: 7603 / 10000
Epoch 5: 7789 / 10000
Epoch 6: 8236 / 10000
Epoch 7: 8355 / 10000
Epoch 8: 8365 / 10000
Epoch 9: 8392 / 10000
Epoch 10: 8362 / 10000
Epoch 11: 8394 / 10000
Epoch 12: 8378 / 10000
Epoch 13: 8393 / 10000
Epoch 14: 8395 / 10000
Epoch 15: 8392 / 10000
Epoch 16: 8390 / 10000
Epoch 17: 8382 / 10000
Epoch 18: 8387 / 10000
Epoch 19: 8410 / 10000
Epoch 20: 8368 / 10000
Epoch 21: 8399 / 10000
Epoch 22: 8401 / 10000
Epoch 23: 8416 / 10000
Epoch 24: 8407 / 10000
Epoch 25: 8419 / 10000
Epoch 26: 8401 / 10000
Epoch 27: 8407 / 10000
Epoch 28: 8411 / 10000
Epoch 29: 8396 / 10000


In [56]:
net = Network([784, 10])
net.SGD(training_data, 30, 10, 1.9, test_data=test_data)

Epoch 0: 5608 / 10000
Epoch 1: 5664 / 10000
Epoch 2: 6589 / 10000
Epoch 3: 6642 / 10000
Epoch 4: 7385 / 10000
Epoch 5: 8277 / 10000
Epoch 6: 8310 / 10000
Epoch 7: 8340 / 10000
Epoch 8: 8366 / 10000
Epoch 9: 8360 / 10000
Epoch 10: 8387 / 10000
Epoch 11: 8369 / 10000
Epoch 12: 8386 / 10000
Epoch 13: 8377 / 10000
Epoch 14: 8386 / 10000
Epoch 15: 8390 / 10000
Epoch 16: 8380 / 10000
Epoch 17: 8401 / 10000
Epoch 18: 8400 / 10000
Epoch 19: 8380 / 10000
Epoch 20: 8401 / 10000
Epoch 21: 8343 / 10000
Epoch 22: 8396 / 10000
Epoch 23: 8395 / 10000
Epoch 24: 8398 / 10000
Epoch 25: 8388 / 10000
Epoch 26: 8397 / 10000
Epoch 27: 8397 / 10000
Epoch 28: 8401 / 10000
Epoch 29: 8405 / 10000
